In [6]:
import time
from datetime import datetime
from DataPreparation import load2d
from SaveModel import save_arch, save_history
from plotter import plot_hist, plot_model_arch
import pickle
from sklearn.cross_validation import train_test_split
from Flip_Image_Generator import FlippedImageDataGenerator
from keras.models import Sequential
from keras.layers import Convolution2D, MaxPooling2D, Flatten, Dense, Activation
from keras.optimizers import SGD
from keras.callbacks import ModelCheckpoint

TabError: inconsistent use of tabs and spaces in indentation (DataPreparation.py, line 61)

In [2]:
model_name = 'model3'
nb_epoch = 3000
validation_split = 0.2
lr = 0.01
momentum = 0.9
nesterov = True
loss_method = 'mean_squared_error'
arch_path = 'model/' + model_name + '-arch-' + str(nb_epoch) + '.json'
weights_path = 'model/' + model_name + '-weights-' + str(nb_epoch) + '.hdf5'


In [3]:
##### load data
X, y = load2d()

NameError: name 'load' is not defined

In [12]:
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=validation_split, random_state=42)

In [4]:
X_flipped = X[:, :, :, ::-1]

def plot_sample(x, y, axis):
    img = x.reshape(96, 96)
    axis.imshow(img, cmap='gray')
    axis.scatter(y[0::2] * 48 + 48, y[1::2] * 48 + 48, marker='x', s=10)
    
fig = pyplot.figure(figsize=(6, 3))
ax = fig.add_subplot(1, 2, 1, xticks=[], yticks=[])
plot_sample(X[1], y[1], ax)
ax = fig.add_subplot(1, 2, 2, xticks=[], yticks=[])
plot_sample(X_flipped[1], y[1], ax)
pyplot.show()

NameError: name 'X' is not defined

In [11]:
flip_indices = [
    (0, 2), (1, 3),
    (4, 8), (5, 9), (6, 10), (7, 11),
    (12, 16), (13, 17), (14, 18), (15, 19),
    (22, 24), (23, 25),
    ]
FTRAIN = 'data/training.csv'
df = read_csv(os.path.expanduser(FTRAIN))
for i, j in flip_indices:
    print("{} -> {}".format(df.columns[i], df.columns[j]))

left_eye_center_x -> right_eye_center_x
left_eye_center_y -> right_eye_center_y
left_eye_inner_corner_x -> right_eye_inner_corner_x
left_eye_inner_corner_y -> right_eye_inner_corner_y
left_eye_outer_corner_x -> right_eye_outer_corner_x
left_eye_outer_corner_y -> right_eye_outer_corner_y
left_eyebrow_inner_end_x -> right_eyebrow_inner_end_x
left_eyebrow_inner_end_y -> right_eyebrow_inner_end_y
left_eyebrow_outer_end_x -> right_eyebrow_outer_end_x
left_eyebrow_outer_end_y -> right_eyebrow_outer_end_y
mouth_left_corner_x -> mouth_right_corner_x
mouth_left_corner_y -> mouth_right_corner_y


In [13]:
from keras import backend as K

def reshape2d_by_image_dim_ordering(X):
    if K.image_dim_ordering() == 'th':
        X = X.reshape(X.shape[0], 1, 96, 96)
        input_shape = (1, 96, 96)
    else:
        X = X.reshape(X.shape[0], 96, 96, 1)
        input_shape = (96, 96, 1)
    
    return X, input_shape

In [14]:
X_train, input_shape = reshape2d_by_image_dim_ordering(X_train)
X_val,   _           = reshape2d_by_image_dim_ordering(X_val)

In [15]:
model = Sequential()

model.add(Convolution2D(32, 3, 3, input_shape = input_shape))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Convolution2D(64, 2, 2))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Convolution2D(128, 2, 2))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Flatten())
model.add(Dense(500))
model.add(Activation('relu'))
model.add(Dense(500))
model.add(Activation('relu'))
model.add(Dense(30))

save_arch(model, arch_path)

In [16]:
checkpoint_collback = ModelCheckpoint(filepath = weights_path,
                                      monitor='val_loss',
                                      save_best_only=True,
                                      mode='auto')

flip_gen = FlippedImageDataGenerator()

sgd = SGD(lr=lr, momentum=momentum, nesterov=nesterov)
model.compile(loss=loss_method, optimizer=sgd)
start_time = time.time()
print('start_time: %s' % (datetime.now()))
hist = model.fit_generator(flip_gen.flow(X_train, y_train),
                           samples_per_epoch=X_train.shape[0],
                           nb_epoch=nb_epoch,
                           validation_data=(X_val, y_val),
                           callbacks=[checkpoint_collback])
print('end_time: %s, duracion(min): %d' % (datetime.now(), int(time.time()-start_time) / 60))

save_history(hist, model_name)

start_time: 2017-03-17 01:18:06.081209
Epoch 1/3000
 256/1712 [===>..........................] - ETA: 43s - loss: 0.1442

KeyboardInterrupt: 